# Map Me If You Can
- Tested with Jupyter version > 5.0 && < 6.0
- Tutorials see [ArcGIS API for Python](https://developers.arcgis.com/python/guide/)

In [1]:
!pip install --user arcgis

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [76]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import pandas

## Connect to ArcGIS Online

In [4]:
geospatial = GIS()

## Show me a map centered on Bonn

In [5]:
bonn_map = geospatial.map("Bonn")
bonn_map

MapView(layout=Layout(height='400px', width='100%'))

## Use the CORONAVIRUS tracker API
- Only world countries or county level for the US

In [88]:
import urllib.request

def extract_from_json(data_frame, parent, child=None):
    if None is child:
        return data_frame.apply(lambda record: record[0][parent], axis=1)
    else:
        return data_frame.apply(lambda record: record[0][parent][child], axis=1)

def tracker_locations_to_dataframe():
    url = "https://coronavirus-tracker-api.herokuapp.com/v2/locations"
    with urllib.request.urlopen(url) as stream:
        locations_complex = pandas.read_json(stream)
        locations = pandas.DataFrame()
        locations["latitude"] = extract_from_json(locations_complex, "coordinates", "latitude").astype(float)
        locations["longitude"] = extract_from_json(locations_complex, "coordinates", "longitude").astype(float)
        locations["country"] = extract_from_json(locations_complex, "country")
        locations["country_code"] = extract_from_json(locations_complex, "country_code")
        locations["last_updated"] = extract_from_json(locations_complex, "last_updated")
        locations["last_updated"] = locations.apply(lambda record: pandas.to_datetime(record["last_updated"]), axis=1)
        return locations
    
def tracker_locations_by_country_code(country_code):
    locations = tracker_locations_to_dataframe()
    return locations.loc[locations["country_code"] == country_code]

def tracker_locations_to_spatial():
    locations = tracker_locations_to_dataframe()
    return GeoAccessor.from_xy(locations, x_column="longitude", y_column="latitude", sr=4326)

In [89]:
tracker_locations = tracker_locations_to_spatial()
world_map = geospatial.map()
tracker_locations.spatial.plot(map_widget=world_map)
world_map

MapView(layout=Layout(height='400px', width='100%'))